# zib gereed maken voor mapping

* inlezen en layout consistent maken met XtEHR
* opslaan voor mapping - zie mapping Notebook

## imports

In [1]:
import pandas as pd
import os

## toon de zibs in folder (hier: zib2024)

In [8]:
base = 'zibs2024'
folder = os.path.join(base, 'downloads')
files = os.listdir(folder)
[(i, file) for i, file in enumerate(files)]

[(0, 'Nl.zorg.Alert-v5.0(2024EN).xlsx'),
 (1, 'Nl.zorg.Condition-v1.1(2024EN).xlsx'),
 (2, 'Nl.zorg.Diagnosis-v2.0(2024EN).xlsx'),
 (3, 'Nl.zorg.HealthcareProvider-v3.6(2024EN).xlsx'),
 (4, 'Nl.zorg.HealthProfessional-v4.0(2024EN).xlsx'),
 (5, 'Nl.zorg.MedicalDevice-v5.0(2024EN).xlsx'),
 (6, 'Nl.zorg.NursingIntervention-v5.0(2024EN).xlsx'),
 (7, 'Nl.zorg.part.AddressInformation-v1.2(2024EN).xlsx'),
 (8, 'Nl.zorg.part.AnatomicalLocation-v1.0.4(2024EN).xlsx'),
 (9, 'Nl.zorg.part.ContactInformation-v1.3.1(2024EN).xlsx'),
 (10, 'Nl.zorg.part.NameInformation-v1.2(2024EN).xlsx'),
 (11, 'Nl.zorg.Patient-v4.3(2024EN).xlsx'),
 (12, 'Nl.zorg.Procedure-v6.0(2024EN).xlsx'),
 (13, 'Nl.zorg.process.RegistrationData-v1.1.2(2024EN).xlsx')]

## kies een zib

In [9]:
file = files[1]
print(file)
url = os.path.join(folder, file)
zibdf = pd.read_excel(url, sheet_name='Data', header=1)
zibdf

Nl.zorg.Condition-v1.1(2024EN).xlsx


c:\Users\MarcdeGraauw\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Concept,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Alias,Type,Card.,Stereotype,Id,Definition,DefinitionCode,Reference,Constraints
0,NaN,Condition,NaN,NaN,NaN,NaN,NaN,NL: AandoeningOfGesteldheid,NaN,NaN,rootconcept,NL-CM:5.4.1,This is a reference to the root concept of inf...,SNOMED CT: 64572001 Disease,NaN,NaN
1,NaN,NaN,PeriodPresent::TimeInterval,NaN,NaN,NaN,NaN,NL: PeriodeAanwezig::TijdsInterval,NaN,1,"data,reference",NL-CM:5.4.8,The period in which the condition is (was) pre...,NaN,This is a reference to the rootconcept of info...,NaN
2,NaN,NaN,StatusDate,NaN,NaN,NaN,NaN,NL: StatusDatum,TS,1,data,NL-CM:5.4.3,The moment from when the data regarding the co...,SNOMED CT: 350351000146106 Date health conditi...,NaN,NaN
3,NaN,NaN,Course,NaN,NaN,NaN,NaN,NL: Beloop,CD,0..1,data,NL-CM:5.4.5,The course of the condition. This is independe...,NaN,CourseCodelist,NaN
4,NaN,NaN,Severity,NaN,NaN,NaN,NaN,NL: Ernst,CD,1,data,NL-CM:5.4.6,Indication of the severity of the condition. A...,SNOMED CT: 246112005 Severity,SeverityCodelist,NaN
5,NaN,NaN,Comment,NaN,NaN,NaN,NaN,NL: Toelichting,ST,0..1,data,NL-CM:5.4.7,A comment in free text with respect to the con...,LOINC: 48767-8 Annotation comment [Interpretat...,NaN,NaN


## opschonen zib

* lowercase kolomnamen
* paar labels laten vervallen
* name en path kolom toevoegen
* name vullen (naam attribuut)
* kolommen opschonen tot benodigde

Handmatig:
* path vullen (als in logical model, dus pad.pad2.pad3.naam etc.)

In [10]:
zibdf.columns = [col.lower() for col in zibdf.columns]
zibdf = zibdf.drop(labels=['unnamed: 0', 'reference'], axis=1).dropna(how='all')
zibdf.insert(0, 'name', None)
zibdf.insert(0, 'path', None)
zibdf.fillna('')
concepts = []
for i, row in zibdf.iterrows():
    # Loop thru columns 2 tot 8 (Concept and 'Unnamed' ones)
    for j in range(2, 6):
        # print(j, row.iloc[j])
        if not pd.isna(row.iloc[j]):
            # Remove last element if next column is empty
            if len(concepts) > j -2:
                concepts.pop()
            try:
                concepts[j - 2] = row.iloc[j]
            except IndexError:
                concepts.append(row.iloc[j])
    # print(i, concepts)
    zibdf.at[i, 'path'] = '.'.join(concepts)
    zibdf.at[i, 'name'] = concepts[-1]
zibdf = zibdf[['name', 'path', 'alias', 'type', 'card.', 'stereotype', 'id', 'definition', 'definitioncode']].fillna('')
zibdf

,name,path,alias,type,card.,stereotype,id,definition,definitioncode
0,Condition,Condition,NL: AandoeningOfGesteldheid,,,rootconcept,NL-CM:5.4.1,This is a reference to the root concept of inf...,SNOMED CT: 64572001 Disease
1,PeriodPresent::TimeInterval,Condition.PeriodPresent::TimeInterval,NL: PeriodeAanwezig::TijdsInterval,,1,"data,reference",NL-CM:5.4.8,The period in which the condition is (was) pre...,
2,StatusDate,Condition.StatusDate,NL: StatusDatum,TS,1,data,NL-CM:5.4.3,The moment from when the data regarding the co...,SNOMED CT: 350351000146106 Date health conditi...
3,Course,Condition.Course,NL: Beloop,CD,0..1,data,NL-CM:5.4.5,The course of the condition. This is independe...,
4,Severity,Condition.Severity,NL: Ernst,CD,1,data,NL-CM:5.4.6,Indication of the severity of the condition. A...,SNOMED CT: 246112005 Severity
5,Comment,Condition.Comment,NL: Toelichting,ST,0..1,data,NL-CM:5.4.7,A comment in free text with respect to the con...,LOINC: 48767-8 Annotation comment [Interpretat...


## Excel opslaan

In [11]:
url = os.path.join(base, 'mappable', file)
zibdf.to_excel(url, index=False)